In [1]:
#we load our libraries
import pandas as pd
import sys
sys.path.append('/Users/halukamb/Documents/CodeRelated/HBDataviz_new/2025/01_BTW100Deutsche/Wrangle/lib/python3.12/site-packages')
import numpy as np
from json import loads, dumps

In [2]:
! pip install pyreadstat


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
#we have one function that filters the ALLBUS data for whether respondents are german and of voting age
def germanElectorate(df):
    german_df=df[df["german"]=="JA"]
    german_df=german_df.dropna(subset="age")
    german_df["age"]=german_df["age"].astype(float)
    german_electorate=german_df[german_df["age"]>=18]
    return german_electorate

In [4]:
#we have another function that regroups the data into fitting categories to the data we have from the zensus
def reformgroups(df):
    df["Alter"]=None
    df["Alter"]=np.where(df["age"]<30,"U30",df["Alter"])
    df["Alter"]=np.where(df["age"]>=30,"30-49",df["Alter"])
    df["Alter"]=np.where(df["age"]>=50,"50-64",df["Alter"])
    df["Alter"]=np.where(df["age"]>64,"Ü65",df["Alter"])
    df["Geschlecht"]=np.nan
    df["Geschlecht"]=np.where(df["sex"]=="FRAU","Weiblich",df["Geschlecht"])
    df["Geschlecht"]=np.where(df["sex"]=="MANN","Männlich",df["Geschlecht"])
    df["OstWest"]=np.nan
    df["OstWest"]=np.where(df["eastwest"]=="ALTE BUNDESLAENDER","West",df["OstWest"])
    df["OstWest"]=np.where(df["eastwest"]=="NEUE BUNDESLAENDER","Ost",df["OstWest"])
    df["OstWest"]=np.where(df["land"].str.contains("BERLIN"),"Berlin",df["OstWest"])

    df["Besiedelung"]=np.nan
    df["Besiedelung"]=np.where(df["gs01"].isin(['GROSSSTADT']),"dicht besiedelt",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['MITTEL-, KLEINSTADT', 'VORORT GROSSSTADT']),"mittlere Besiedlungsdichte",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['LAENDL. DORF','EINZELHAUS, LAND']),"gering besiedelt",df["Besiedelung"])
    
    


    
    return df

In [5]:
#and lastly we have a stratification function that takes the zensus data and the skewed dataset of allbus
#  and creates a stratified dataset based on the population shares of the zensus data
#for each combination of age, gender, eastwest and population density
def stratifier(zensus_model,skewed_df):

    stratified_dataset=pd.DataFrame()
    samplesize_sum=0

    #we go through each row of the zensus groups
    for index, row in zensus_model.iterrows():
        #take in which share of the population this group represents
        samplesize = round(row["Share"] * 100)
        samplesize_sum += samplesize
        #and then we take a copy of the skewed allbus data
        sub_df = skewed_df.copy()
        
        for i in range(0, 4):
            #and then we filter it down, feature by feature of the speficic row of the zensus data
            #we do this from 0 to 4 because in the first four rows we have the features we want to stratify by
            sub_df = sub_df[sub_df[row.index[i]] == row.values[i]]
            print(row.index[i],row.values[i],len(sub_df))
        
        
        #we draw a sample from this group that correlates to the share of the population
        #if the original dataset is too small, we sample with replacement
        sampled_subgroup = sub_df.sample(samplesize, random_state=1, replace=True)
        #and add this sampled group to the overall stratified dataset
        stratified_dataset = pd.concat([stratified_dataset, sampled_subgroup])
    print(len(stratified_dataset),samplesize_sum)
    return stratified_dataset


In [6]:
#this is the zensus data we need, we remove all rows with NaN values
#as we need as detailed data as possible for the stratification
zensus_model=pd.read_csv("zensus_model.csv")
zensus_model=zensus_model.dropna()
zensus_model=zensus_model.sort_values(by="Share",ascending=False)


In [7]:
#we read in the allbus data for 2021
df_2021 = pd.read_spss("../data/AllbusData/ZA5280_v2-0-1.sav")
# filter it for the german electorate

df_2021 = germanElectorate(df_2021)
# regroup it to fit the zensus data

df_2021= reformgroups(df_2021)
# and stratify it based on the zensus data
df_2021= stratifier(zensus_model,df_2021)
df_2021

Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter Ü65 223
Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter 50-64 247
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter 50-64 233
Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter 30-49 193
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter 30-49 176
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter Ü65 263
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter Ü65 60
Geschlecht Männlich 2419
Besiedelung dicht besiedelt 451
OstWest West 247
Alter 30-49 79
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter 30-49 83
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter 50-64 60
Geschlecht Männlich 2419
Besiedelung d

,za_nr,doi,version,respid,substudy,mode,splt21,eastwest,german,ep01,...,bik,gkpol,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
986,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),987.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1 ;>499.999,100.000 - 499.999 E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3459,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3460.0,SIMULTAN,MAIL,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;<500.000,100.000 - 499.999 E.,ALTE BUNDESLAENDER,NaN,NaN,NaN,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1790,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1791.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,GUT,...,ZONE 2-4;>499.999,20.000 - 49.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3381,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3382.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
4800,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4801.0,SEQUENZIELL,CAWI,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 2-4;<500.000,5.000 - 19.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
1464,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1465.0,SIMULTAN,MAIL,SPLIT B,NEUE BUNDESLAENDER,JA,SCHLECHT,...,ZONE 2-4;<500.000,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
2291,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),2292.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;>499.999,5.000 - 19.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt


In [8]:
#we read in the allbus data for 2023
df_2023 = pd.read_spss("../data/AllbusData/ZA8830_v1-0-0.sav")
# filter it for the german electorate

df_2023 = germanElectorate(df_2023)
# regroup it to fit the zensus data

df_2023= reformgroups(df_2023)
#and straitify it based on the zensus data
df_2023= stratifier(zensus_model,df_2023)
df_2023


Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter Ü65 208
Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter 50-64 230
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter 50-64 184
Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter 30-49 197
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter 30-49 178
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter Ü65 222
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter Ü65 76
Geschlecht Männlich 2292
Besiedelung dicht besiedelt 471
OstWest West 257
Alter 30-49 85
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter 30-49 60
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter 50-64 64
Geschlecht Männlich 2292
Besiedelung d

,za_nr,doi,version,respid,substudy,mode,splt23_1,splt23_2,eastwest,german,...,bik,xs11,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
770,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",771.0,NaN,CAPI,NaN,SPLIT A,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,183.0,1.252326,1.69639,1.242308,2.107439,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3538,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3539.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,93.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1776,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1777.0,SIMULTAN,MAIL,SPLIT B,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1-4;<50.000,10.0,1.203727,1.693259,1.192166,2.018646,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3432,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3433.0,SEQUENZIELL,CAWI,SPLIT A,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,168.0,1.207467,0.554394,1.205184,0.668147,Ü65,Weiblich,West,mittlere Besiedlungsdichte
5150,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",5151.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,143.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1898.0,NaN,CAPI,NaN,SPLIT B,NEUE BUNDESLAENDER,JA,...,5.000-19.999 EINW.,55.0,0.501346,0.532744,0.524886,0.27963,U30,Weiblich,Ost,gering besiedelt
3272,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3273.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,176.0,0.544816,0.845091,0.572064,0.483446,U30,Weiblich,Ost,gering besiedelt
4876,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",4877.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,31.0,0.544816,0.338036,0.572064,0.193378,U30,Weiblich,Ost,gering besiedelt
4876,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",4877.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,31.0,0.544816,0.338036,0.572064,0.193378,U30,Weiblich,Ost,gering besiedelt


In [29]:
#this is a function that enables us to analyze for each for each subgroup how the respondents answered differently
#  to a given question/variable
def analyze_factor_patterns(dataset,variable_to_check):
    #we have a base dataframe that holds all results
    base_df=pd.DataFrame()
    #now we define the four features we want to use to check for changes in the respondents data in allbus
    factor_list=["Alter","Geschlecht","OstWest","Besiedelung"]
    #we iterate through all possible combinations of these features
    for element in factor_list:
        #print(element)
        for element2 in factor_list:
            #add we za_nr to the list of the variable to check, as it is the unique identifier for each respondent
            baselist=[variable_to_check,"za_nr"]

            #we build also a list of the features we want to check for
            extralist=[element,element2]
            #set it to remove duplicates, in case we pick the same feature twice
            extralist=list(set(extralist))

            #and combine the two lists and filter the dataset for the columns we need
            baselist.extend(extralist)
            dataset_reduced=dataset[baselist]
            #remove all rows with missing values
            dataset_reduced = dataset_reduced[~dataset_reduced.isin(["nan"]).any(axis=1)]

            #we also define a extralist plus which we need to group by the dataset
            #it is the name of the features we want to check for plus the variable
            # and then we count
            extralist_plus_factor=extralist+[variable_to_check]
            grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
            #we remove all columns that have missing values
            grouped_df=grouped_df.dropna(axis=1)
            #and then we calculate the share of the respondents for each subgroup by using the sum of the group
            grouped_df[variable_to_check+'_percent'] = grouped_df['za_nr']*100 / grouped_df.groupby(by=extralist)['za_nr'].transform('sum')
            grouped_df[variable_to_check+'_absolute'] = grouped_df['za_nr']
            
            #and we add this to the base dataframe that holds all results
            grouped_df=grouped_df.reset_index()
            base_df=pd.concat([base_df,grouped_df],axis=0)
    #at the end we drop duplicates
    base_df=base_df.drop_duplicates()
    #reduce the dataset
    base_df_cols=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', variable_to_check,
       variable_to_check+'_absolute',variable_to_check+'_percent']
    #and return the dataset
    return base_df[base_df_cols]

In [30]:
def compare_years_var(factor_to_check,df_2021,df_2023):
    #we analyze the factor patterns for the first year
    df_2021_analyzed=analyze_factor_patterns(df_2021,factor_to_check)
    #and we analyze the factor patterns for the second year
    df_2023_analyzed=analyze_factor_patterns(df_2023,factor_to_check)
    #we merge the two datasets
    merged_df=pd.merge(df_2021_analyzed,df_2023_analyzed,on=["Alter", "Besiedelung", "Geschlecht", "OstWest", factor_to_check],suffixes=('_2021', '_2023'))
    merged_df=merged_df[merged_df["OstWest"]!="Berlin"]

    #and return the merged dataset

    return merged_df

In [38]:
factor_to_check="mc03"
comparison_df_kontakt_auslaender_nachbarn=compare_years_var(factor_to_check,df_2021,df_2023)
comparison_df_kontakt_auslaender_nachbarn=comparison_df_kontakt_auslaender_nachbarn[comparison_df_kontakt_auslaender_nachbarn[factor_to_check]=='JA']
comparison_df_kontakt_auslaender_nachbarn=comparison_df_kontakt_auslaender_nachbarn[comparison_df_kontakt_auslaender_nachbarn.columns[comparison_df_kontakt_auslaender_nachbarn.columns!=factor_to_check]]

comparison_df_kontakt_auslaender_nachbarn=comparison_df_kontakt_auslaender_nachbarn.reset_index(drop=True)
comparison_df_kontakt_auslaender_nachbarn

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to

,Alter,Besiedelung,Geschlecht,OstWest,mc03_absolute_2021,mc03_percent_2021,mc03_absolute_2023,mc03_percent_2023
0,30-49,NaN,NaN,NaN,1138,65.065752,1192,63.370548
1,50-64,NaN,NaN,NaN,1113,63.202726,1128,58.841941
2,U30,NaN,NaN,NaN,557,59.956943,568,55.092144
3,Ü65,NaN,NaN,NaN,928,52.727273,911,45.963673
4,30-49,NaN,Männlich,NaN,612,70.425777,624,67.168999
5,50-64,NaN,Männlich,NaN,570,65.592635,589,61.675393
6,U30,NaN,Männlich,NaN,258,55.010661,292,55.619048
7,Ü65,NaN,Männlich,NaN,379,51.917808,441,49.718151
8,30-49,NaN,Weiblich,NaN,526,59.772727,568,59.663866
9,50-64,NaN,Weiblich,NaN,543,60.874439,539,56.029106


In [39]:
factor_to_check="ma01b"
comparison_df_auslaender_anpassen=compare_years_var(factor_to_check,df_2021,df_2023)
comparison_df_auslaender_anpassen=comparison_df_auslaender_anpassen[comparison_df_auslaender_anpassen[factor_to_check]=='STIMME VOLL ZU']
comparison_df_auslaender_anpassen=comparison_df_auslaender_anpassen[comparison_df_auslaender_anpassen.columns[comparison_df_auslaender_anpassen.columns!=factor_to_check]]
comparison_df_auslaender_anpassen=comparison_df_auslaender_anpassen.reset_index(drop=True)
comparison_df_auslaender_anpassen

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to

,Alter,Besiedelung,Geschlecht,OstWest,ma01b_absolute_2021,ma01b_percent_2021,ma01b_absolute_2023,ma01b_percent_2023
0,30-49,NaN,NaN,NaN,373,21.314286,361,19.335833
1,50-64,NaN,NaN,NaN,484,27.114846,607,31.713689
2,U30,NaN,NaN,NaN,91,9.660297,127,12.129895
3,Ü65,NaN,NaN,NaN,883,46.449237,853,41.793239
4,30-49,NaN,Männlich,NaN,166,18.693694,183,20.065789
5,50-64,NaN,Männlich,NaN,265,29.642058,336,35.257083
6,U30,NaN,Männlich,NaN,51,10.714286,65,11.992620
7,Ü65,NaN,Männlich,NaN,359,45.966709,332,36.563877
8,30-49,NaN,Weiblich,NaN,207,24.013921,178,18.638743
9,50-64,NaN,Weiblich,NaN,219,24.579125,271,28.199792


In [40]:
factor_to_check="va03"
comparison_df_inflation=compare_years_var(factor_to_check,df_2021,df_2023)
comparison_df_inflation=comparison_df_inflation[comparison_df_inflation["va03"].isin(['AM WICHTIGSTEN', 'AM ZWEITWICHTIGSTEN'])]

comparison_df_inflation=comparison_df_inflation[comparison_df_inflation.columns[comparison_df_inflation.columns!=factor_to_check]]
comparison_df_inflation=comparison_df_inflation.groupby(by=["Alter","Geschlecht","OstWest","Besiedelung"], dropna=False).sum().reset_index()

comparison_df_inflation

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to

,Alter,Geschlecht,OstWest,Besiedelung,va03_absolute_2021,va03_percent_2021,va03_absolute_2023,va03_percent_2023
0,30-49,Männlich,NaN,NaN,227,24.620390,471,33.933718
1,30-49,Weiblich,NaN,NaN,200,22.497188,639,46.950771
2,30-49,NaN,Ost,NaN,75,23.809524,199,44.320713
3,30-49,NaN,West,NaN,328,23.213022,857,39.493088
4,30-49,NaN,NaN,dicht besiedelt,146,21.661721,363,35.449219
5,30-49,NaN,NaN,gering besiedelt,120,30.456853,257,46.057348
6,30-49,NaN,NaN,mittlere Besiedlungsdichte,161,21.668910,490,41.988003
7,30-49,NaN,NaN,NaN,427,23.578134,1110,40.378319
8,50-64,Männlich,NaN,NaN,123,12.799168,339,24.266285
9,50-64,Weiblich,NaN,NaN,147,16.647792,446,31.879914


In [45]:
comp_df_1=pd.merge(comparison_df_auslaender_anpassen,comparison_df_kontakt_auslaender_nachbarn,on=["Alter", "Besiedelung", "Geschlecht", "OstWest"])
comp_df_2=pd.merge(comp_df_1,comparison_df_inflation,on=["Alter", "Besiedelung", "Geschlecht", "OstWest"])
comp_df_vars=comp_df_2[comp_df_2.columns[~comp_df_2.columns.str.contains("absolute")]]
comp_df_vars

,Alter,Besiedelung,Geschlecht,OstWest,ma01b_percent_2021,ma01b_percent_2023,mc03_percent_2021,mc03_percent_2023,va03_percent_2021,va03_percent_2023
0,30-49,NaN,NaN,NaN,21.314286,19.335833,65.065752,63.370548,23.578134,40.378319
1,50-64,NaN,NaN,NaN,27.114846,31.713689,63.202726,58.841941,14.642082,28.075823
2,U30,NaN,NaN,NaN,9.660297,12.129895,59.956943,55.092144,22.293676,44.444444
3,Ü65,NaN,NaN,NaN,46.449237,41.793239,52.727273,45.963673,17.889391,25.135233
4,30-49,NaN,Männlich,NaN,18.693694,20.065789,70.425777,67.168999,24.620390,33.933718
5,50-64,NaN,Männlich,NaN,29.642058,35.257083,65.592635,61.675393,12.799168,24.266285
6,U30,NaN,Männlich,NaN,10.714286,11.992620,55.010661,55.619048,17.987152,39.917695
7,Ü65,NaN,Männlich,NaN,45.966709,36.563877,51.917808,49.718151,18.104496,23.284314
8,30-49,NaN,Weiblich,NaN,24.013921,18.638743,59.772727,59.663866,22.497188,46.950771
9,50-64,NaN,Weiblich,NaN,24.579125,28.199792,60.874439,56.029106,16.647792,31.879914


In [46]:
def party_comparison():
    #also we do the same for party preferences
    df_2021_party=analyze_factor_patterns(df_2021,"pv01")
    df_2023_party=analyze_factor_patterns(df_2023,"pv01")
    #we also merge the datasets of 2021 and 2023 based on all the features for the party preferences
    comp_party=pd.merge(df_2021_party,df_2023_party,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'pv01'],suffixes=["_2021","_2023"])
    comp_party["diff_percent"]=comp_party["percent_2023"]-comp_party["percent_2021"]
    comp_party.sort_values("diff_percent")

    #and we pivot it based on the party preferences
    comp_party_pivot=comp_party.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='pv01')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()
    return comp_party_pivot

In [53]:
df_2023_party=analyze_factor_patterns(df_2023,"pv01")
df_2021_party=analyze_factor_patterns(df_2021,"pv01")


#we also merge the datasets of 2021 and 2023 based on all the features for the party preferences
comp_party=pd.merge(df_2021_party,df_2023_party,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'pv01'],suffixes=["_2021","_2023"])
comp_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_68644/842391348.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to

,Alter,Besiedelung,Geschlecht,OstWest,pv01,pv01_absolute_2021,pv01_percent_2021,pv01_absolute_2023,pv01_percent_2023
0,30-49,NaN,NaN,NaN,AFD,137,6.135244,271,12.212708
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,114,5.105240,77,3.470032
2,30-49,NaN,NaN,NaN,CDU-CSU,477,21.361397,426,19.197837
3,30-49,NaN,NaN,NaN,DIE GRUENEN,743,33.273623,534,24.064894
4,30-49,NaN,NaN,NaN,DIE LINKE,143,6.403941,138,6.219018
...,...,...,...,...,...,...,...,...,...
515,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,881,25.403691,711,19.921547
516,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,200,5.767013,158,4.427010
517,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,471,13.581315,381,10.675259
518,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,537,15.484429,721,20.201737


In [67]:

#and we pivot it based on the party preferences
comp_party_pivot=comp_party.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='pv01')[['pv01_percent_2021','pv01_percent_2023']].reset_index()
comp_party_pivot

Alter                 Besiedelung Geschlecht OstWest pv01_percent_2021  \
pv01                                                                    AFD   
0      NaN                         NaN        NaN  Berlin          2.950820   
1      NaN                         NaN        NaN     Ost         11.746522   
2      NaN                         NaN        NaN    West          4.289587   
3      NaN                         NaN   Männlich     NaN          7.808564   
4      NaN                         NaN   Männlich  Berlin          4.827586   
..     ...                         ...        ...     ...               ...   
60     Ü65                         NaN   Männlich     NaN          6.226766   
61     Ü65                         NaN   Weiblich     NaN          1.906275   
62     Ü65             dicht besiedelt        NaN     NaN          3.819918   
63     Ü65            gering besiedelt        NaN     NaN          5.040323   
64     Ü65  mittlere Besiedlungsdichte        NaN     NaN          3.435805   

                                                                            \
pv01 ANDERE PARTEI    CDU-CSU DIE GRUENEN  DIE LINKE        FDP        SPD   
0         3.606557  21.639344   26.885246   9.180328   7.213115  19.672131   
1         5.641422  23.106646   14.992272  12.905719  12.210201  12.287481   
2         3.412534  25.912933   29.835752   5.469622  11.880083  15.324510   
3         3.803526  24.055416   24.382872   7.506297  13.173804  15.566751   
4         2.068966  17.931034   27.586207   7.586207  12.413793  18.620690   
..             ...        ...         ...        ...        ...        ...   
60        0.743494  30.762082   14.962825   5.947955  13.661710  24.814126   
61        1.985703  36.219222   17.553614   6.592534  10.484512  19.618745   
62        1.227831  30.422920   17.462483   4.911323   8.867667  27.557981   
63        2.016129  35.483871   13.709677   8.266129  11.693548  19.354839   
64        1.265823  35.081374   16.817360   6.329114  14.104882  19.529837   

                          pv01_percent_2023                           \
pv01 WUERDE NICHT WAEHLEN               AFD ANDERE PARTEI    CDU-CSU   
0                8.852459          6.528190      1.780415  13.353116   
1                7.109737         22.702703      3.783784  20.617761   
2                3.874980          7.488300      3.354134  25.101404   
3                3.702771         12.895317      3.824467  23.044864   
4                8.965517          9.202454      1.840491  12.269939   
..                    ...               ...           ...        ...   
60               2.881041          7.732865      0.702988  33.831283   
61               5.639396          4.384615      1.769231  32.769231   
62               5.729877          3.926380      0.736196  28.098160   
63               4.435484          9.255533      2.615694  38.832998   
64               3.435805          5.950266      1.065719  34.547069   

                                                                        
pv01 DIE GRUENEN  DIE LINKE        FDP        SPD WUERDE NICHT WAEHLEN  
0      24.925816  23.145401   5.637982  17.804154             6.824926  
1      11.274131  10.810811   5.945946  19.459459             5.405405  
2      23.681747   3.744150  10.000000  21.669267             4.960998  
3      18.632018   5.516058  10.835989  20.004903             5.246384  
4      22.085890  25.153374   4.294479  17.791411             7.361963  
..           ...        ...        ...        ...                  ...  
60     13.532513   4.745167   8.084359  28.207381             3.163445  
61     16.846154   3.153846   7.230769  29.846154             4.000000  
62     12.392638   4.539877  10.306748  35.828221             4.171779  
63     13.078471   4.828974   4.426559  23.541247             3.420523  
64     18.383659   3.019538   7.104796  26.642984             3.285968  

[65 rows x 20 columns]

In [68]:
# to remove the double index ot the columns, we bring them to one name in one line
new_columns=[]
for index in comp_party_pivot:

    if index[1]!="":
        print(index[1])
        var_name=index[0].replace("pv01_","")
        new_columns.append(index[1]+"_"+var_name)
    else:
        new_columns.append(index[0])
new_columns
comp_party_pivot.columns=new_columns
comp_party_pivot

AFD
ANDERE PARTEI
CDU-CSU
DIE GRUENEN
DIE LINKE
FDP
SPD
WUERDE NICHT WAEHLEN
AFD
ANDERE PARTEI
CDU-CSU
DIE GRUENEN
DIE LINKE
FDP
SPD
WUERDE NICHT WAEHLEN


,Alter,Besiedelung,Geschlecht,OstWest,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,DIE LINKE_percent_2021,FDP_percent_2021,SPD_percent_2021,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023
0,NaN,NaN,NaN,Berlin,2.950820,3.606557,21.639344,26.885246,9.180328,7.213115,19.672131,8.852459,6.528190,1.780415,13.353116,24.925816,23.145401,5.637982,17.804154,6.824926
1,NaN,NaN,NaN,Ost,11.746522,5.641422,23.106646,14.992272,12.905719,12.210201,12.287481,7.109737,22.702703,3.783784,20.617761,11.274131,10.810811,5.945946,19.459459,5.405405
2,NaN,NaN,NaN,West,4.289587,3.412534,25.912933,29.835752,5.469622,11.880083,15.324510,3.874980,7.488300,3.354134,25.101404,23.681747,3.744150,10.000000,21.669267,4.960998
3,NaN,NaN,Männlich,NaN,7.808564,3.803526,24.055416,24.382872,7.506297,13.173804,15.566751,3.702771,12.895317,3.824467,23.044864,18.632018,5.516058,10.835989,20.004903,5.246384
4,NaN,NaN,Männlich,Berlin,4.827586,2.068966,17.931034,27.586207,7.586207,12.413793,18.620690,8.965517,9.202454,1.840491,12.269939,22.085890,25.153374,4.294479,17.791411,7.361963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,6.226766,0.743494,30.762082,14.962825,5.947955,13.661710,24.814126,2.881041,7.732865,0.702988,33.831283,13.532513,4.745167,8.084359,28.207381,3.163445
61,Ü65,NaN,Weiblich,NaN,1.906275,1.985703,36.219222,17.553614,6.592534,10.484512,19.618745,5.639396,4.384615,1.769231,32.769231,16.846154,3.153846,7.230769,29.846154,4.000000
62,Ü65,dicht besiedelt,NaN,NaN,3.819918,1.227831,30.422920,17.462483,4.911323,8.867667,27.557981,5.729877,3.926380,0.736196,28.098160,12.392638,4.539877,10.306748,35.828221,4.171779
63,Ü65,gering besiedelt,NaN,NaN,5.040323,2.016129,35.483871,13.709677,8.266129,11.693548,19.354839,4.435484,9.255533,2.615694,38.832998,13.078471,4.828974,4.426559,23.541247,3.420523


In [69]:
#and then again we merge both datasets
comp_factors_party_pivot=pd.merge(comp_df_vars,comp_party_pivot,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_factors_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,ma01b_percent_2021,ma01b_percent_2023,mc03_percent_2021,mc03_percent_2023,va03_percent_2021,va03_percent_2023,...,SPD_percent_2021,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023
0,30-49,NaN,NaN,NaN,21.314286,19.335833,65.065752,63.370548,23.578134,40.378319,...,10.031348,4.612629,12.212708,3.470032,19.197837,24.064894,6.219018,10.184768,17.755746,6.894998
1,50-64,NaN,NaN,NaN,27.114846,31.713689,63.202726,58.841941,14.642082,28.075823,...,15.216380,4.374128,12.818222,5.314873,23.760607,21.214828,5.270210,8.173292,17.686467,5.761501
2,U30,NaN,NaN,NaN,9.660297,12.129895,59.956943,55.092144,22.293676,44.444444,...,9.973981,5.464007,8.115183,3.752182,13.263525,31.937173,9.336824,12.390925,17.626527,3.577661
3,Ü65,NaN,NaN,NaN,46.449237,41.793239,52.727273,45.963673,17.889391,25.135233,...,22.012848,4.368308,5.947498,1.271534,33.264971,15.299426,3.896637,7.629204,29.081214,3.609516
4,30-49,NaN,Männlich,NaN,18.693694,20.065789,70.425777,67.168999,24.620390,33.933718,...,9.300266,3.897254,14.623838,4.311074,19.949281,17.666948,5.494505,11.073542,19.019442,7.861369
5,50-64,NaN,Männlich,NaN,29.642058,35.257083,65.592635,61.675393,12.799168,24.266285,...,16.402116,3.968254,17.500000,6.034483,20.775862,20.517241,5.086207,10.000000,15.086207,5.000000
6,U30,NaN,Männlich,NaN,10.714286,11.992620,55.010661,55.619048,17.987152,39.917695,...,9.508716,4.278922,10.367893,4.515050,13.043478,26.588629,7.859532,17.224080,15.886288,4.515050
7,Ü65,NaN,Männlich,NaN,45.966709,36.563877,51.917808,49.718151,18.104496,23.284314,...,24.814126,2.881041,7.732865,0.702988,33.831283,13.532513,4.745167,8.084359,28.207381,3.163445
8,30-49,NaN,Weiblich,NaN,24.013921,18.638743,59.772727,59.663866,22.497188,46.950771,...,10.778986,5.344203,9.459459,2.509653,18.339768,31.370656,7.046332,9.169884,16.312741,5.791506
9,50-64,NaN,Weiblich,NaN,24.579125,28.199792,60.874439,56.029106,16.647792,31.879914,...,13.891626,4.827586,7.784986,4.541242,26.969416,21.964782,5.468026,6.209453,20.481928,6.580167


In [70]:
comp_factors_party_pivot.columns

Index(['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'ma01b_percent_2021',
       'ma01b_percent_2023', 'mc03_percent_2021', 'mc03_percent_2023',
       'va03_percent_2021', 'va03_percent_2023', 'AFD_percent_2021',
       'ANDERE PARTEI_percent_2021', 'CDU-CSU_percent_2021',
       'DIE GRUENEN_percent_2021', 'DIE LINKE_percent_2021',
       'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
       'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
       'FDP_percent_2023', 'SPD_percent_2023',
       'WUERDE NICHT WAEHLEN_percent_2023'],
      dtype='object')

In [71]:
#we take in the data from the zensus that gives a share of the population for each subgroup
#but only with two features defined as the same time
zensus_two_level=pd.read_csv("json_none.csv")
zensus_two_level

,Geschlecht,Besiedelung,OstWest,Alter,Share
0,NaN,NaN,Ost,U30,1.675804
1,NaN,gering besiedelt,NaN,U30,2.543912
2,NaN,dicht besiedelt,Ost,NaN,3.853220
3,NaN,NaN,Ost,30-49,4.559863
4,NaN,NaN,Ost,50-64,4.751534
5,NaN,NaN,Ost,Ü65,5.482466
6,NaN,gering besiedelt,Ost,NaN,5.491600
7,NaN,gering besiedelt,NaN,30-49,5.723338
8,NaN,mittlere Besiedlungsdichte,NaN,U30,5.842770
9,NaN,dicht besiedelt,NaN,U30,6.005464


In [72]:
# we merge this
comp_factors_party_pivot_zensus=pd.merge(comp_factors_party_pivot,zensus_two_level,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])


In [73]:
#and we filter only for the percentage results  
perc_share_df=comp_factors_party_pivot_zensus[comp_factors_party_pivot_zensus.columns[~comp_factors_party_pivot_zensus.columns.str.contains("absolute")]]

In [74]:
perc_share_df

,Alter,Besiedelung,Geschlecht,OstWest,ma01b_percent_2021,ma01b_percent_2023,mc03_percent_2021,mc03_percent_2023,va03_percent_2021,va03_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,30-49,NaN,NaN,NaN,21.314286,19.335833,65.065752,63.370548,23.578134,40.378319,...,4.612629,12.212708,3.470032,19.197837,24.064894,6.219018,10.184768,17.755746,6.894998,27.801470
1,50-64,NaN,NaN,NaN,27.114846,31.713689,63.202726,58.841941,14.642082,28.075823,...,4.374128,12.818222,5.314873,23.760607,21.214828,5.270210,8.173292,17.686467,5.761501,28.653535
2,U30,NaN,NaN,NaN,9.660297,12.129895,59.956943,55.092144,22.293676,44.444444,...,5.464007,8.115183,3.752182,13.263525,31.937173,9.336824,12.390925,17.626527,3.577661,14.392146
3,Ü65,NaN,NaN,NaN,46.449237,41.793239,52.727273,45.963673,17.889391,25.135233,...,4.368308,5.947498,1.271534,33.264971,15.299426,3.896637,7.629204,29.081214,3.609516,29.096249
4,30-49,NaN,Männlich,NaN,18.693694,20.065789,70.425777,67.168999,24.620390,33.933718,...,3.897254,14.623838,4.311074,19.949281,17.666948,5.494505,11.073542,19.019442,7.861369,13.956047
5,50-64,NaN,Männlich,NaN,29.642058,35.257083,65.592635,61.675393,12.799168,24.266285,...,3.968254,17.500000,6.034483,20.775862,20.517241,5.086207,10.000000,15.086207,5.000000,14.218163
6,U30,NaN,Männlich,NaN,10.714286,11.992620,55.010661,55.619048,17.987152,39.917695,...,4.278922,10.367893,4.515050,13.043478,26.588629,7.859532,17.224080,15.886288,4.515050,7.343611
7,Ü65,NaN,Männlich,NaN,45.966709,36.563877,51.917808,49.718151,18.104496,23.284314,...,2.881041,7.732865,0.702988,33.831283,13.532513,4.745167,8.084359,28.207381,3.163445,12.741598
8,30-49,NaN,Weiblich,NaN,24.013921,18.638743,59.772727,59.663866,22.497188,46.950771,...,5.344203,9.459459,2.509653,18.339768,31.370656,7.046332,9.169884,16.312741,5.791506,13.845423
9,50-64,NaN,Weiblich,NaN,24.579125,28.199792,60.874439,56.029106,16.647792,31.879914,...,4.827586,7.784986,4.541242,26.969416,21.964782,5.468026,6.209453,20.481928,6.580167,14.435372


In [75]:
perc_share_df=perc_share_df.round(1)
perc_share_df


,Alter,Besiedelung,Geschlecht,OstWest,ma01b_percent_2021,ma01b_percent_2023,mc03_percent_2021,mc03_percent_2023,va03_percent_2021,va03_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,30-49,NaN,NaN,NaN,21.3,19.3,65.1,63.4,23.6,40.4,...,4.6,12.2,3.5,19.2,24.1,6.2,10.2,17.8,6.9,27.8
1,50-64,NaN,NaN,NaN,27.1,31.7,63.2,58.8,14.6,28.1,...,4.4,12.8,5.3,23.8,21.2,5.3,8.2,17.7,5.8,28.7
2,U30,NaN,NaN,NaN,9.7,12.1,60.0,55.1,22.3,44.4,...,5.5,8.1,3.8,13.3,31.9,9.3,12.4,17.6,3.6,14.4
3,Ü65,NaN,NaN,NaN,46.4,41.8,52.7,46.0,17.9,25.1,...,4.4,5.9,1.3,33.3,15.3,3.9,7.6,29.1,3.6,29.1
4,30-49,NaN,Männlich,NaN,18.7,20.1,70.4,67.2,24.6,33.9,...,3.9,14.6,4.3,19.9,17.7,5.5,11.1,19.0,7.9,14.0
5,50-64,NaN,Männlich,NaN,29.6,35.3,65.6,61.7,12.8,24.3,...,4.0,17.5,6.0,20.8,20.5,5.1,10.0,15.1,5.0,14.2
6,U30,NaN,Männlich,NaN,10.7,12.0,55.0,55.6,18.0,39.9,...,4.3,10.4,4.5,13.0,26.6,7.9,17.2,15.9,4.5,7.3
7,Ü65,NaN,Männlich,NaN,46.0,36.6,51.9,49.7,18.1,23.3,...,2.9,7.7,0.7,33.8,13.5,4.7,8.1,28.2,3.2,12.7
8,30-49,NaN,Weiblich,NaN,24.0,18.6,59.8,59.7,22.5,47.0,...,5.3,9.5,2.5,18.3,31.4,7.0,9.2,16.3,5.8,13.8
9,50-64,NaN,Weiblich,NaN,24.6,28.2,60.9,56.0,16.6,31.9,...,4.8,7.8,4.5,27.0,22.0,5.5,6.2,20.5,6.6,14.4


In [76]:
perc_share_df["Geschlecht"]=perc_share_df["Geschlecht"].str.replace("Männlich","Maennlich")
perc_share_df["Alter"]=perc_share_df["Alter"].str.replace("Ü65","65plus")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("dicht besiedelt","Stadt")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("gering besiedelt","Dorf")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("mittlere Besiedlungsdichte","Kleinstadt")

In [77]:
#and we write this into a csv file
perc_share_df.to_csv("perc_share_df_new.csv")

In [79]:
perc_share_df.columns

Index(['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'ma01b_percent_2021',
       'ma01b_percent_2023', 'mc03_percent_2021', 'mc03_percent_2023',
       'va03_percent_2021', 'va03_percent_2023', 'AFD_percent_2021',
       'ANDERE PARTEI_percent_2021', 'CDU-CSU_percent_2021',
       'DIE GRUENEN_percent_2021', 'DIE LINKE_percent_2021',
       'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
       'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
       'FDP_percent_2023', 'SPD_percent_2023',
       'WUERDE NICHT WAEHLEN_percent_2023', 'Share'],
      dtype='object')

In [81]:
variables_dict={"lebensstil_auslaender":"ma01b","kontakt_auslaender":"mc03","inflation":"va03"}

In [89]:
json_array=[]
for index, row in perc_share_df.iterrows():
    print(row)
    base_dict=row[["Alter","Besiedelung","Geschlecht","OstWest","Share"]].to_dict()
    print(base_dict)
    for key in variables_dict.keys():
        var_2023=row[variables_dict[key]+"_percent_2023"]
        var_2021=row[variables_dict[key]+"_percent_2021"]
        innerdict={"percent_2021":var_2021,"percent_2023":var_2023}
        base_dict[key]=innerdict
    base_dict["wahl"]=row[['AFD_percent_2021', 'ANDERE PARTEI_percent_2021',
        'CDU-CSU_percent_2021', 'DIE GRUENEN_percent_2021',
        'DIE LINKE_percent_2021', 'FDP_percent_2021', 'SPD_percent_2021',
        'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
        'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
        'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
        'FDP_percent_2023', 'SPD_percent_2023',
        'WUERDE NICHT WAEHLEN_percent_2023']].to_dict()
    json_array.append(base_dict)
json_array[0]

Alter                                30-49
Besiedelung                            NaN
Geschlecht                             NaN
OstWest                                NaN
ma01b_percent_2021                    21.3
ma01b_percent_2023                    19.3
mc03_percent_2021                     65.1
mc03_percent_2023                     63.4
va03_percent_2021                     23.6
va03_percent_2023                     40.4
AFD_percent_2021                       6.1
ANDERE PARTEI_percent_2021             5.1
CDU-CSU_percent_2021                  21.4
DIE GRUENEN_percent_2021              33.3
DIE LINKE_percent_2021                 6.4
FDP_percent_2021                      13.1
SPD_percent_2021                      10.0
WUERDE NICHT WAEHLEN_percent_2021      4.6
AFD_percent_2023                      12.2
ANDERE PARTEI_percent_2023             3.5
CDU-CSU_percent_2023                  19.2
DIE GRUENEN_percent_2023              24.1
DIE LINKE_percent_2023                 6.2
FDP_percent

{'Alter': '30-49',
 'Besiedelung': nan,
 'Geschlecht': nan,
 'OstWest': nan,
 'Share': 27.8,
 'lebensstil_auslaender': {'percent_2021': 21.3, 'percent_2023': 19.3},
 'kontakt_auslaender': {'percent_2021': 65.1, 'percent_2023': 63.4},
 'inflation': {'percent_2021': 23.6, 'percent_2023': 40.4},
 'wahl': {'AFD_percent_2021': 6.1,
  'ANDERE PARTEI_percent_2021': 5.1,
  'CDU-CSU_percent_2021': 21.4,
  'DIE GRUENEN_percent_2021': 33.3,
  'DIE LINKE_percent_2021': 6.4,
  'FDP_percent_2021': 13.1,
  'SPD_percent_2021': 10.0,
  'WUERDE NICHT WAEHLEN_percent_2021': 4.6,
  'AFD_percent_2023': 12.2,
  'ANDERE PARTEI_percent_2023': 3.5,
  'CDU-CSU_percent_2023': 19.2,
  'DIE GRUENEN_percent_2023': 24.1,
  'DIE LINKE_percent_2023': 6.2,
  'FDP_percent_2023': 10.2,
  'SPD_percent_2023': 17.8,
  'WUERDE NICHT WAEHLEN_percent_2023': 6.9}}

In [90]:
json_array[1]

{'Alter': '50-64',
 'Besiedelung': nan,
 'Geschlecht': nan,
 'OstWest': nan,
 'Share': 28.7,
 'lebensstil_auslaender': {'percent_2021': 27.1, 'percent_2023': 31.7},
 'kontakt_auslaender': {'percent_2021': 63.2, 'percent_2023': 58.8},
 'inflation': {'percent_2021': 14.6, 'percent_2023': 28.1},
 'wahl': {'AFD_percent_2021': 6.0,
  'ANDERE PARTEI_percent_2021': 3.6,
  'CDU-CSU_percent_2021': 27.4,
  'DIE GRUENEN_percent_2021': 27.9,
  'DIE LINKE_percent_2021': 6.0,
  'FDP_percent_2021': 9.5,
  'SPD_percent_2021': 15.2,
  'WUERDE NICHT WAEHLEN_percent_2021': 4.4,
  'AFD_percent_2023': 12.8,
  'ANDERE PARTEI_percent_2023': 5.3,
  'CDU-CSU_percent_2023': 23.8,
  'DIE GRUENEN_percent_2023': 21.2,
  'DIE LINKE_percent_2023': 5.3,
  'FDP_percent_2023': 8.2,
  'SPD_percent_2023': 17.7,
  'WUERDE NICHT WAEHLEN_percent_2023': 5.8}}

In [32]:
! pip3 install simplejson


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [91]:
import simplejson as json
#And then we write out the file as a json file
with open('data_for_viz_more_factors.json', 'w', encoding='utf-8') as file:
    json.dump(json_array, file, ignore_nan=True, indent=2)


In [94]:
perc_share_df

,Alter,Besiedelung,Geschlecht,OstWest,ma01b_percent_2021,ma01b_percent_2023,mc03_percent_2021,mc03_percent_2023,va03_percent_2021,va03_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,30-49,NaN,NaN,NaN,21.3,19.3,65.1,63.4,23.6,40.4,...,4.6,12.2,3.5,19.2,24.1,6.2,10.2,17.8,6.9,27.8
1,50-64,NaN,NaN,NaN,27.1,31.7,63.2,58.8,14.6,28.1,...,4.4,12.8,5.3,23.8,21.2,5.3,8.2,17.7,5.8,28.7
2,U30,NaN,NaN,NaN,9.7,12.1,60.0,55.1,22.3,44.4,...,5.5,8.1,3.8,13.3,31.9,9.3,12.4,17.6,3.6,14.4
3,65plus,NaN,NaN,NaN,46.4,41.8,52.7,46.0,17.9,25.1,...,4.4,5.9,1.3,33.3,15.3,3.9,7.6,29.1,3.6,29.1
4,30-49,NaN,Maennlich,NaN,18.7,20.1,70.4,67.2,24.6,33.9,...,3.9,14.6,4.3,19.9,17.7,5.5,11.1,19.0,7.9,14.0
5,50-64,NaN,Maennlich,NaN,29.6,35.3,65.6,61.7,12.8,24.3,...,4.0,17.5,6.0,20.8,20.5,5.1,10.0,15.1,5.0,14.2
6,U30,NaN,Maennlich,NaN,10.7,12.0,55.0,55.6,18.0,39.9,...,4.3,10.4,4.5,13.0,26.6,7.9,17.2,15.9,4.5,7.3
7,65plus,NaN,Maennlich,NaN,46.0,36.6,51.9,49.7,18.1,23.3,...,2.9,7.7,0.7,33.8,13.5,4.7,8.1,28.2,3.2,12.7
8,30-49,NaN,Weiblich,NaN,24.0,18.6,59.8,59.7,22.5,47.0,...,5.3,9.5,2.5,18.3,31.4,7.0,9.2,16.3,5.8,13.8
9,50-64,NaN,Weiblich,NaN,24.6,28.2,60.9,56.0,16.6,31.9,...,4.8,7.8,4.5,27.0,22.0,5.5,6.2,20.5,6.6,14.4
